# Image processing for timelapse blob images
-> script part to add to timelapse part for real time processing

In [ ]:
import matplotlib.image as mpimg
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#access images - to adapt
# to work directly on the image that is being recorded
%cd images-originals/
%ls
img_path = 'tl10-168.jpg'
orig_img = cv2.imread(img_path)
# plt.imshow(orig_img)

In [ ]:
#if conversion to RGB (but not useful as we conv to gray)
#BGR_img = cv2.imread(img_path)
#orig_img = cv2.cvtColor(BGR_img, cv2.COLOR_BGR2RGB)

# conversion to grayscale
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#global adjustment of brightness and contrast
alpha = 1   #contrast
beta = 40   #brightness
img = cv2.convertScaleAbs(orig_img, alpha=alpha, beta=beta)
plt.imshow(img)

#save - find again how to 
cv2.imwrite("grayscale.jpg", gr_img)

In [ ]:
#contour - edge detection and drawing
#to test : is it too heavy to run on RaspPi ?
"""
Canny:
The Process of Canny edge detection algorithm can be broken down to 5 different steps:
1. Apply Gaussian filter to smooth the image in order to remove the noise
2. Find the intensity gradients of the image
3. Apply non-maximum suppression to get rid of spurious response to edge detection
4. Apply double threshold to determine potential edges (supplied by the user)
5. Track edge by hysteresis: Finalize the detection of edges by suppressing all the other edges that
are weak and not connected to strong edges.
"""
#Autocanny
sigma = 0.3
median = np.median(img)

# apply automatic Canny edge detection using the computed median
lower = int(max(0, (1.0 - sigma) * median))
upper = int(min(255, (1.0 + sigma) * median))
img = cv2.Canny(img, lower, upper)

#cv2.imshow("Auto Canny", auto_canny) #indeed much better than Canny directly applied
cv2.waitKey(0)
cv2.destroyAllWindows()


Option : Linear / radial gradient to homogeneize the led ring light
the part below runs, but it's very long (why ? the pixel by pixel grayscale conversion above goes fast...)

In [ ]:
from __future__ import print_function
from builtins import input
import cv2 as cv
import numpy as np
import argparse

# Read image given by user
#parser = argparse.ArgumentParser(description='Code for Changing the contrast and brightness of an image! tutorial.')
#parser.add_argument('--input', help='Path to input image.', default='lena.jpg')
#args = parser.parse_args()
#image = cv.imread(cv.samples.findFile(args.input))
image = orig_img
if image is None:
    print('Could not open or find the image: ', args.input)
    exit(0)
new_image = np.zeros(image.shape, image.dtype)

# Initialize values
print(' Basic Linear Transforms ')
print('-------------------------')

new_image = orig_img
alpha = 1   #contrast
beta = 70   #brightness
img_len = image.shape[1]

# Linear gradient to adjust brightness
# Do the operation new_image(i,j) = alpha*image(i,j) + beta
#accessing each pixel separately
for y in range(image.shape[0]):
    for x in range(0, image.shape[1]):
        beta_x = beta - ((x+1)*beta/img_len)
        new_image[y,x] = np.clip(alpha*image[y,x] + beta_x, 0, 255)
#cv.imshow('Original Image', image)
#cv.imshow('New Image', new_image)

# create figure
fig = plt.figure(figsize=(20, 7))
rows = 1
columns = 2
# Adds a subplot at the 1st position
fig.add_subplot(rows, columns, 1)
plt.imshow(orig_img, cmap='gray')
plt.axis('off')
plt.title("original")
# Adds a subplot at the 1st position
fig.add_subplot(rows, columns, 2)
plt.imshow(new_image, cmap='gray')
plt.axis('off')
plt.title("adjusted")

# Wait until user press some key
#cv.waitKey()

In [ ]:
# Radial gradient to adjust brightness
# seems to work but very slow, is there a way to optimize it ?

import numpy as np
import cv2
   
     
#reading the image
input_image = cv2.imread('food.jpeg')
   
#resizing the image according to our need
# resize() function takes 2 parameters, 
# the image and the dimensions
input_image = cv2.resize(input_image, (480, 480))
   
# Extracting the height and width of an image
rows, cols = input_image.shape[:2]
   
# generating vignette mask using Gaussian
# resultant_kernels
X_resultant_kernel = cv2.getGaussianKernel(cols,200)
Y_resultant_kernel = cv2.getGaussianKernel(rows,200)
   
#generating resultant_kernel matrix
resultant_kernel = Y_resultant_kernel * X_resultant_kernel.T
   
#creating mask and normalising by using np.linalg
# function
mask = 255 * resultant_kernel / np.linalg.norm(resultant_kernel)
output = np.copy(input_image)
   
# applying the mask to each channel in the input image
for i in range(3):
    output[:,:,i] = output[:,:,i] * mask
       
#displaying the original image  
cv2.imshow('Original', input_image)
   
#displaying the vignette filter image
cv2.imshow('VIGNETTE', output)
   
# Maintain output window utill
# user presses a key
cv2.waitKey(0)
   
# Destroying present windows on screen
cv2.destroyAllWindows()
